## Load Model

In [ ]:
pip install langchain_together

In [ ]:
import pandas as pd
import time
import copy
import os
from langchain_together import ChatTogether
api_key = "api"

def load_model(prompting_type, model_name="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free", temperature=1, top_p=1.0, max_output_tokens=512):
    model = ChatTogether(
            model=model_name,
            temperature=temperature,
            max_tokens=max_output_tokens,
            top_p=top_p,
            together_api_key=api_key
        )
    return model

## Load Dataset

In [ ]:
def load_dataset(task):
    """
    Load dataset for a given task.
    """
    task1 = pd.read_csv("../BenNumEval_run_1_random_1000/Task1_random_150.csv")
    task2 = pd.read_csv("../BenNumEval_run_1_random_1000/Task2_random_250.csv")
    task3 = pd.read_csv("../BenNumEval_run_1_random_1000/Task3_random_150.csv")
    task4 = pd.read_csv("../BenNumEval_run_1_random_1000/Task4_random_150.csv")
    task5 = pd.read_csv("../BenNumEval_run_1_random_1000/Task5_random_150.csv")
    task6 = pd.read_csv("../BenNumEval_run_1_random_1000/Task6_random_150.csv")

    task_dict = {
        "task1": task1,
        "task2": task2,
        "task3": task3,
        "task4": task4,
        "task5": task5,
        "task6": task6
    }
    return task_dict.get(task, "Invalid task")

## Prompts

In [ ]:
# Prompt templates
xlp_prompt_template_task1246 = (
    "# Instructions:\n"
    "You are a Math Expert AI model proficient in both Bengali and English. Your task is to solve a mathematical reasoning problem provided in Bengali and deliver the solution in English.\n\n"
    "Please adhere to the following format:\n"
    "1. Translate the Bengali problem into English for clarity.\n"
    "2. Conclude with the final numerical answer, formatted as **\"Answer: [num]\"**.\n\n"
    "# Problem: {question}\n\n"
    "# Response:"
)

xcot_prompt_template_task1246 = (
    "# Instructions:\n"
    "You are a Math Expert AI model proficient in both Bengali and English. Your task is to solve a mathematical reasoning problem provided in Bengali and deliver the solution in English.\n\n"
    "Please adhere to the following format:\n"
    "1. Translate the Bengali problem into English for clarity.\n"
    "2. Provide a detailed step-by-step solution with explanations in English, following the logical flow of reasoning.\n"
    "3. Conclude with the final numerical answer, formatted as **\"Answer: [num]\"**.\n\n"
    "# Problem: {question}\n\n"
    "# Response:"
)

bnap_prompt_template_task1246 = (
    "# নির্দেশাবলী:\n"
    "আপনি একজন গণিত বিশেষজ্ঞ এআই মডেল, যিনি বাংলা ভাষায় সম্পূর্ণভাবে দক্ষ। আপনার কাজ হলো প্রদত্ত গাণিতিক সমস্যার বিশদভাবে সমাধান করা এবং উত্তরটি বাংলায় প্রদান করা।\n"
    "আপনাকে অবশ্যই চূড়ান্ত সাংখ্যিক উত্তরটি নিম্নলিখিত ফরম্যাটে উপস্থাপন করতে হবে: **\"উত্তর: [সংখ্যা]\"**।\n\n"
    "# সমস্যা: {question}\n\n"
    "# সমাধান:"
)

xlp_prompt_template_task3 = (
    "# Instructions:\n"
    "You are a Math Expert AI model proficient in both Bengali and English. Your task is to solve a mathematical reasoning problem provided in Bengali and choose the correct option.\n\n"
    "Please adhere to the following format:\n"
    "1. Translate the Bengali question and options into English for clarity.\n"
    "2. Conclude by selecting the correct option, formatted as **\"Answer: [Option]\"**. The possible options are **\"Option 1\"** or **\"Option 2\"**.\n\n"
    "# Question: {question}\n"
    "# Option 1: {option1}\n"
    "# Option 2: {option2}\n\n"
    "# Response:"
)

xcot_prompt_template_task3 = (
    "# Instructions:\n"
    "You are a Math Expert AI model proficient in both Bengali and English. Your task is to solve a mathematical reasoning problem provided in Bengali and choose the correct option.\n\n"
    "Please adhere to the following format:\n"
    "1. Translate the Bengali question and options into English for clarity.\n"
    "2. Provide a detailed step-by-step solution with explanations in English, following the logical flow of reasoning.\n"
    "3. Conclude by selecting the correct option, formatted as **\"Answer: [Option]\"**. The possible options are **\"Option 1\"** or **\"Option 2\"**.\n\n"
    "# Question: {question}\n"
    "# Option 1: {option1}\n"
    "# Option 2: {option2}\n\n"
    "# Response:"
)


bnap_prompt_template_task3 = (
    "# নির্দেশাবলী:\n"
    "আপনি একজন দক্ষ গণিত বিশেষজ্ঞ এআই, যিনি বাংলা ভাষায় পারদর্শী। আপনার কাজ হলো প্রদত্ত গাণিতিক সমস্যার বিশ্লেষণ করে সঠিক উত্তর নির্বাচন করা।\n\n"
    "আপনার উত্তর অবশ্যই *নির্দিষ্ট বিন্যাসে* প্রদান করতে হবে:\n"
    "**\"উত্তর: [সঠিক সম্ভাব্য উত্তর]\"**\n" 
    "যেখানে **[সঠিক সম্ভাব্য উত্তর]** হবে **\"উত্তর ১\"** অথবা **\"উত্তর ২\"**।\n\n"
    "---\n" 
    "# সমস্যা: {question}\n"
    "# সম্ভাব্য উত্তরসমূহ:\n"
    "# সম্ভাব্য উত্তর ১: {option1}\n"
    "# সম্ভাব্য উত্তর ২: {option2}\n"  
    "---\n\n" 
    "# সমাধান:"
    )


xlp_prompt_template_task5 = (
    "# Instructions:\n"
    "You are a Math Expert AI model proficient in both Bengali and English. Your task is to solve a Quantitative Natural Language Inference (QNLI) problem presented in Bengali. You need to determine the relationship between the premise and the hypothesis.\n\n"
    "Please adhere to the following format:\n"
    "1. Translate both the premise and hypothesis from Bengali to English for clarity.\n"
    "2. Conclude by selecting the correct option, formatted as **\"Answer: [Option]\"**. The possible options are **\"Entailment\"**, **\"Neutral\"**, or **\"Contradiction\"**.\n\n"
    "# Premise: {premise}\n"
    "# Hypothesis: {hypothesis}\n\n"
    "# Response:"
)

xcot_prompt_template_task5 = (
    "# Instructions:\n"
    "You are a Math Expert AI model proficient in both Bengali and English. Your task is to solve a Quantitative Natural Language Inference (QNLI) problem presented in Bengali. You need to determine the relationship between the premise and the hypothesis.\n\n"
    "Please follow these steps:\n"
    "1. Translate both the premise and hypothesis from Bengali to English for clarity.\n"
    "2. Provide a step-by-step explanation of your reasoning process.\n"
    "3. Conclude by selecting the correct option, formatted as **\"Answer: [Option]\"**. The possible options are **\"Entailment\"**, **\"Neutral\"**, or **\"Contradiction\"**.\n\n"
    "# Premise: {premise}\n"
    "# Hypothesis: {hypothesis}\n\n"
    "# Response:"
)

bnap_prompt_template_task5 = (
    "# নির্দেশাবলী:\n"
    "আপনি একজন গণিত বিশেষজ্ঞ AI মডেল, যিনি বাংলা ভাষায় দক্ষ। আপনার কাজ হলো একটি গাণিতিক ভাষাগত অনুমান সমস্যার সমাধান করা। আপনাকে প্রদত্ত পূর্বধারণা ও অনুমান এর মধ্যে সম্পর্ক নির্ধারণ করতে হবে।\n\n"
    "অনুগ্রহ করে নিম্নলিখিত বিন্যাস অনুসরণ করুন:\n"
    "প্রদত্ত পূর্বধারণা ও অনুমান এর মধ্যে সম্পর্ক বিষয়ে চূড়ান্ত সিদ্ধান্ত নিন এবং সঠিক উত্তরটি **\"উত্তর: [সম্ভাব্য সঠিক উত্তর]\"** এই ভাবে প্রদান করুন। উত্তরের সম্ভাব্য বিকল্পগুলি হল: **\"সমর্থন\"**, **\"নিরপেক্ষ\"**, অথবা **\"বিরোধ\"**।\n\n"
    "# পূর্বধারণা: {premise}\n"
    "# অনুমান: {hypothesis}\n\n"
    "# সমাধান:"
)

# Map prompt template names to their content
prompt_map = {
    "xlp_prompt_template_task1246": xlp_prompt_template_task1246,
    "xcot_prompt_template_task1246": xcot_prompt_template_task1246,
    "bnap_prompt_template_task1246": bnap_prompt_template_task1246,
    "xlp_prompt_template_task3": xlp_prompt_template_task3,
    "xcot_prompt_template_task3": xcot_prompt_template_task3,
    "bnap_prompt_template_task3": bnap_prompt_template_task3,
    "xlp_prompt_template_task5": xlp_prompt_template_task5,
    "xcot_prompt_template_task5": xcot_prompt_template_task5,
    "bnap_prompt_template_task5": bnap_prompt_template_task5,
}

## Inference

In [ ]:
import time
prompts = ["xlp", "xcot", "bnap"]
model_name = "DeepSeek-R1-Distill-Llama-70B-free"

rate_limit = 6.0  # queries per minute
sleep_interval = 60.0 / rate_limit  # 10 seconds per query

def process_prompts_for_task(task_id, data):
    """
    Process a given task with the provided dataset and for each prompt type.
    """
    for prompt in prompts:
        responses = []
        chat_histories = []
        num_instances = len(data)
        print(f"Processing Task {task_id} with prompt {prompt} for {num_instances} instances...")

        for i in range(num_instances):
            start_time = time.time()
            # Prepare variables for formatting prompt based on task type
            if task_id in [1, 2, 4, 6]:
                question = data.iloc[i]['Question']
                formatted_prompt = prompt_map.get(f"{prompt}_prompt_template_task1246").format(question=question)
            elif task_id == 3:
                question = data.iloc[i]['Question']
                option1 = data.iloc[i]['option1']
                option2 = data.iloc[i]['option2']
                formatted_prompt = prompt_map.get(f"{prompt}_prompt_template_task{task_id}").format(
                    question=question, option1=option1, option2=option2
                )
            elif task_id == 5:
                # For Task 5, data contains 'Premise' and 'Hypothesis'
                premise = data.iloc[i]['Premise']
                hypothesis = data.iloc[i]['Hypothesis']
                formatted_prompt = prompt_map.get(f"{prompt}_prompt_template_task{task_id}").format(
                    premise=premise, hypothesis=hypothesis
                )
            else:
                print(f"Invalid task id: {task_id}")
                continue

            # Start a new chat session and send the formatted prompt
            model = load_model(prompting_type=prompt)
            response = model.predict(formatted_prompt)
            responses.append(response)

            print(f"\033[91mTask {task_id} | Prompt {prompt} | Question No: {i}\033[0m")
            
            # Calculate elapsed time and sleep for the remaining time to complete 10 seconds.
            elapsed_time = time.time() - start_time
            remaining_sleep = sleep_interval - elapsed_time
            if remaining_sleep > 0:
                print(f"Elapsed time: {elapsed_time:.2f}s | Remaining sleep: {remaining_sleep:.2f}s")
                time.sleep(remaining_sleep)
                print("Sleeping for", remaining_sleep, "seconds...")
        
        # Prepare the dataframe to save the responses and chat histories
        data_with_solutions = copy.deepcopy(data)
        data_with_solutions["Model Response"] = responses
        
        # Save the responses to a CSV file
        output_filename = f"{model_name}_{prompt}_task{task_id}_random_{num_instances}_responses.csv"
        output_path = os.path.join("..", "Model Responses","DeepSeek_R1_Distill_Llama_70B", str(output_filename))
        data_with_solutions.to_csv(output_path, index=False)
        print(f"Saved responses to {output_path}")
        
        # Optionally, print a sample response (if exists)
        print(data_with_solutions.iloc[10]["Model Response"])
        
        # print("Sleeping for 5 seconds before next prompt...")
        # time.sleep(5)
    
    print("Completed all prompts for Task", task_id)
    # print("Sleeping for 5 seconds before next task...")
    # time.sleep(5)

In [ ]:
def process_task(task_id):
    """
    Load dataset for the task and process based on task type.
    """
    dataset_key = f"task{task_id}"
    data = load_dataset(dataset_key)
    if isinstance(data, str):
        print(f"Error loading dataset for task: {task_id}")
        return

    # For tasks 1, 2, 4, 6, 3, and 5 the processing is handled inside process_prompts_for_task.
    process_prompts_for_task(task_id, data)

def main():
    for task in [6, 5, 4, 3, 2, 1]:
        print(f"Starting processing for Task {task}")
        process_task(task)
    print("All tasks processed.")

if __name__ == "__main__":
    main()

Starting processing for Task 6
Processing Task 6 with prompt xlp for 150 instances...


C:\Users\ACER\AppData\Local\Temp\ipykernel_21516\2631702786.py:45: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = model.predict(formatted_prompt)


Task 6 | Prompt xlp | Question No: 0
Elapsed time: 9.17s | Remaining sleep: 0.83s
Sleeping for 0.830054759979248 seconds...
Task 6 | Prompt xlp | Question No: 1
Task 6 | Prompt xlp | Question No: 2
Task 6 | Prompt xlp | Question No: 3
Elapsed time: 8.35s | Remaining sleep: 1.65s
Sleeping for 1.649550199508667 seconds...
Task 6 | Prompt xlp | Question No: 4
Elapsed time: 5.16s | Remaining sleep: 4.84s
Sleeping for 4.8367698192596436 seconds...
Task 6 | Prompt xlp | Question No: 5
Elapsed time: 5.29s | Remaining sleep: 4.71s
Sleeping for 4.713590383529663 seconds...
Task 6 | Prompt xlp | Question No: 6
Elapsed time: 6.14s | Remaining sleep: 3.86s
Sleeping for 3.8592090606689453 seconds...
Task 6 | Prompt xlp | Question No: 7
Task 6 | Prompt xlp | Question No: 8
Task 6 | Prompt xlp | Question No: 9
Task 6 | Prompt xlp | Question No: 10
Task 6 | Prompt xlp | Question No: 11
Elapsed time: 9.34s | Remaining sleep: 0.66s
Sleeping for 0.6630775928497314 seconds...
Task 6 | Prompt xlp | Questio